### More Testing on Multiple Linear Regression Models

Import modules

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

Load in games data and transform to create efensive yards allowed data 

In [2]:
pd.set_option('display.max_columns', 500)


In [3]:
all_games_df = pd.DataFrame(data = pd.read_csv('all_games_data.csv', index_col=0))
field_pos_df = pd.DataFrame(data = pd.read_csv('all_games_field_pos_data.csv', index_col=0))
recruit_df = pd.DataFrame(data = pd.read_csv('all_teams_recruit_data.csv', index_col=0))

clean_games_df = pd.DataFrame(data = pd.read_csv('clean_games_with_opp.csv', index_col=0))

If I ever am able to get all the field position stuff then it can go here

In [4]:
# final_df = pd.merge(clean_games_df, field_pos_df,  how='left', left_on= ['id','school'], right_on = ['game_id','team'])
# final_df = final_df.drop(columns=['game_id','team'])
# final_df.head()

In [5]:
clean_games_df.reset_index(drop=True, inplace=True)
clean_games_df.head()

,school,school_id,points,fumblesRecovered,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,kickingPoints,interceptionYards,interceptionTDs,passesIntercepted,firstDowns,thirdDownEff,fourthDownEff,totalYards,netPassingYards,completionAttempts,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,totalPenaltiesYards,turnovers,fumblesLost,interceptions,possessionTime,puntReturnYards,puntReturnTDs,puntReturns,totalFumbles,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected,possession_minutes,possession_seconds,poss_total_sec,id,year,week,opposing_netPassingYards,opposing_yardsPerPass,opposing_rushingYards,opposing_yardsPerRushAttempt,opposing_passingTDs,opposing_rushingTDs,opposing_points,side,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
0,Navy,2426,17,0.0,2.0,0.0,64.0,0.0,4.0,5.0,0.0,0.0,1.0,22.0,0.333333,1.0,390.0,20.0,2-4,5.0,370.0,63.0,5.9,3-20,1.0,1.0,0.0,32:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,26.0,1946.0,400547982,2014,1,226.0,15.1,194.0,4.8,2.0,1.0,34,A,4.0,12.0,1.0,1.0,2.0,4.0,0.500000
1,UAB,5,48,1.0,4.0,2.0,64.0,0.0,2.0,12.0,NaN,NaN,NaN,25.0,0.642857,NaN,490.0,152.0,13-20,7.6,338.0,55.0,6.1,3-35,2.0,1.0,1.0,30:17,14.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,17.0,1817.0,400548010,2014,1,201.0,6.5,71.0,2.0,0.0,1.0,10,A,9.0,14.0,0.0,0.0,13.0,20.0,0.650000
2,UMass,113,7,0.0,0.0,1.0,95.0,0.0,4.0,1.0,20.0,0.0,1.0,9.0,0.272727,0.0,202.0,147.0,9-22,6.7,55.0,22.0,2.5,4-23,1.0,0.0,1.0,17:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,49.0,1069.0,400547728,2014,1,173.0,6.9,338.0,5.5,1.0,2.0,30,A,3.0,11.0,1.0,1.0,9.0,22.0,0.409091
3,UCF,2116,24,1.0,2.0,1.0,142.0,0.0,5.0,6.0,0.0,0.0,2.0,11.0,0.384615,0.5,246.0,222.0,12-22,10.1,24.0,29.0,0.8,8-47,1.0,1.0,0.0,25:27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,27.0,1527.0,400547642,2014,1,454.0,9.7,57.0,2.0,1.0,1.0,26,A,5.0,13.0,2.0,2.0,12.0,22.0,0.545455
4,Houston,248,7,1.0,1.0,0.0,89.0,0.0,6.0,1.0,NaN,NaN,NaN,18.0,0.250000,0.5,208.0,234.0,25-50,4.7,-26.0,23.0,-1.1,9-47,6.0,2.0,4.0,22:14,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,14.0,1334.0,400547644,2014,1,121.0,5.0,142.0,2.8,0.0,3.0,27,A,4.0,16.0,4.0,4.0,25.0,50.0,0.500000


In [ ]:
# Used this to see how any columns had nulls and how many nulls
# clean_games_df.isna().sum() 

In [6]:
# Dropping all columns with more than 25 nulls, then dropping the individual rows that have nulls
x = clean_games_df.isna().sum().to_dict()
column_drop = []

for k, v in x.items():
    if v > 25 :
        column_drop.append(k)

# Additionally, these columns are weird formats or already covered by other variables (ie the possession stuff)
extend_list = ('completionAttempts','totalPenaltiesYards','possessionTime','possession_minutes','possession_seconds','year','week','school','school_id')

column_drop.extend(extend_list)

clean_games_df.drop(columns=column_drop, inplace = True)
clean_games_df.dropna(axis = 0, inplace = True)

In [7]:
train_density = 0.8
games_ids = set(clean_games_df["id"])
train_ids = random.sample(list(games_ids), int(train_density*len(games_ids)))
test_ids = list(games_ids - set(train_ids))

Extracting Test and Train Games

In [76]:
test_games_df = clean_games_df[clean_games_df["id"].isin(test_ids)]
train_games_df = clean_games_df[clean_games_df["id"].isin(train_ids)]
train_games_df.head()

,points,fumblesRecovered,rushingTDs,passingTDs,firstDowns,thirdDownEff,totalYards,netPassingYards,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,turnovers,fumblesLost,interceptions,poss_total_sec,id,opposing_netPassingYards,opposing_yardsPerPass,opposing_rushingYards,opposing_yardsPerRushAttempt,opposing_passingTDs,opposing_rushingTDs,opposing_points,side,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
1,48,1.0,4.0,2.0,25.0,0.642857,490.0,152.0,7.6,338.0,55.0,6.1,2.0,1.0,1.0,1817.0,400548010,201.0,6.5,71.0,2.0,0.0,1.0,10,A,9.0,14.0,0.0,0.0,13.0,20.0,0.650000
2,7,0.0,0.0,1.0,9.0,0.272727,202.0,147.0,6.7,55.0,22.0,2.5,1.0,0.0,1.0,1069.0,400547728,173.0,6.9,338.0,5.5,1.0,2.0,30,A,3.0,11.0,1.0,1.0,9.0,22.0,0.409091
4,7,1.0,1.0,0.0,18.0,0.250000,208.0,234.0,4.7,-26.0,23.0,-1.1,6.0,2.0,4.0,1334.0,400547644,121.0,5.0,142.0,2.8,0.0,3.0,27,A,4.0,16.0,4.0,4.0,25.0,50.0,0.500000
5,45,2.0,3.0,3.0,31.0,0.500000,574.0,313.0,6.7,261.0,50.0,5.2,2.0,1.0,1.0,1806.0,400547648,91.0,2.3,-24.0,-1.0,0.0,0.0,0,A,10.0,20.0,2.0,2.0,27.0,47.0,0.574468
6,10,2.0,1.0,0.0,23.0,0.470588,355.0,284.0,5.9,71.0,31.0,2.3,2.0,1.0,1.0,2012.0,400547641,308.0,8.6,205.0,5.5,3.0,2.0,35,A,8.0,17.0,4.0,4.0,25.0,48.0,0.520833


In [77]:
test_games_df.keys()

Index(['points', 'fumblesRecovered', 'rushingTDs', 'passingTDs', 'firstDowns',
       'thirdDownEff', 'totalYards', 'netPassingYards', 'yardsPerPass',
       'rushingYards', 'rushingAttempts', 'yardsPerRushAttempt', 'turnovers',
       'fumblesLost', 'interceptions', 'poss_total_sec', 'id',
       'opposing_netPassingYards', 'opposing_yardsPerPass',
       'opposing_rushingYards', 'opposing_yardsPerRushAttempt',
       'opposing_passingTDs', 'opposing_rushingTDs', 'opposing_points', 'side',
       'thirdDownConverts', 'thirdDownAttempts', 'fourthDownConverts',
       'fourthDownAttempts', 'completions', 'passAttempts',
       'passCompletionPercentage'],
      dtype='object')

In [78]:
train_games_df

,points,fumblesRecovered,rushingTDs,passingTDs,firstDowns,thirdDownEff,totalYards,netPassingYards,yardsPerPass,rushingYards,rushingAttempts,yardsPerRushAttempt,turnovers,fumblesLost,interceptions,poss_total_sec,id,opposing_netPassingYards,opposing_yardsPerPass,opposing_rushingYards,opposing_yardsPerRushAttempt,opposing_passingTDs,opposing_rushingTDs,opposing_points,side,thirdDownConverts,thirdDownAttempts,fourthDownConverts,fourthDownAttempts,completions,passAttempts,passCompletionPercentage
1,48,1.0,4.0,2.0,25.0,0.642857,490.0,152.0,7.6,338.0,55.0,6.1,2.0,1.0,1.0,1817.0,400548010,201.0,6.5,71.0,2.0,0.0,1.0,10,A,9.0,14.0,0.0,0.0,13.0,20.0,0.650000
2,7,0.0,0.0,1.0,9.0,0.272727,202.0,147.0,6.7,55.0,22.0,2.5,1.0,0.0,1.0,1069.0,400547728,173.0,6.9,338.0,5.5,1.0,2.0,30,A,3.0,11.0,1.0,1.0,9.0,22.0,0.409091
4,7,1.0,1.0,0.0,18.0,0.250000,208.0,234.0,4.7,-26.0,23.0,-1.1,6.0,2.0,4.0,1334.0,400547644,121.0,5.0,142.0,2.8,0.0,3.0,27,A,4.0,16.0,4.0,4.0,25.0,50.0,0.500000
5,45,2.0,3.0,3.0,31.0,0.500000,574.0,313.0,6.7,261.0,50.0,5.2,2.0,1.0,1.0,1806.0,400547648,91.0,2.3,-24.0,-1.0,0.0,0.0,0,A,10.0,20.0,2.0,2.0,27.0,47.0,0.574468
6,10,2.0,1.0,0.0,23.0,0.470588,355.0,284.0,5.9,71.0,31.0,2.3,2.0,1.0,1.0,2012.0,400547641,308.0,8.6,205.0,5.5,3.0,2.0,35,A,8.0,17.0,4.0,4.0,25.0,48.0,0.520833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16210,26,3.0,2.0,0.0,12.0,0.200000,213.0,147.0,4.9,66.0,34.0,1.9,0.0,0.0,0.0,2192.0,401539480,120.0,3.8,35.0,1.5,0.0,0.0,0,H,3.0,15.0,1.0,1.0,22.0,30.0,0.733333
16211,26,0.0,1.0,1.0,21.0,0.375000,396.0,203.0,6.2,193.0,45.0,4.3,3.0,1.0,2.0,2047.0,401539477,238.0,6.6,31.0,1.2,1.0,1.0,14,H,6.0,16.0,1.0,1.0,19.0,33.0,0.575758
16212,31,0.0,1.0,3.0,17.0,0.300000,363.0,239.0,7.0,124.0,20.0,6.2,1.0,0.0,1.0,1372.0,401539475,324.0,7.9,157.0,4.2,2.0,2.0,34,H,3.0,10.0,2.0,2.0,21.0,34.0,0.617647
16213,23,1.0,3.0,0.0,22.0,0.437500,383.0,275.0,7.1,108.0,41.0,2.6,2.0,2.0,0.0,2129.0,401539484,192.0,7.7,271.0,7.7,1.0,5.0,49,H,7.0,16.0,3.0,3.0,28.0,39.0,0.717949


Test and Train data

In [79]:
train_ids

[401426599,
 400869290,
 401114346,
 401415243,
 401282676,
 401112168,
 401532608,
 400787478,
 400547906,
 400869287,
 401117493,
 400933876,
 400787981,
 400937525,
 401531392,
 400941839,
 401403969,
 401520188,
 400548168,
 400548357,
 400937488,
 401404157,
 401110816,
 401112216,
 401282211,
 401110799,
 400935301,
 401110826,
 400935398,
 401405084,
 400757018,
 401310733,
 401282816,
 400933841,
 401013035,
 400547665,
 400763651,
 400935285,
 400547896,
 401426592,
 401524025,
 400756935,
 401282125,
 400868925,
 400869565,
 401520196,
 401281951,
 401022526,
 401114300,
 400933892,
 401411123,
 401015049,
 401056699,
 401282154,
 401415241,
 400869821,
 401531432,
 401411166,
 400548374,
 401121972,
 400548329,
 401207185,
 400756886,
 400941815,
 400869516,
 400547829,
 401114293,
 400547792,
 400548138,
 400763467,
 401112263,
 400937499,
 401015000,
 400945013,
 401112497,
 401117898,
 400935385,
 401112496,
 401237146,
 400935374,
 400869552,
 401520182,
 400869641,
 401

In [80]:
train_games_df.side.value_counts()['H']
# 6476
train_games_df.side.value_counts()['A']
# 6478

test_games_df.side.value_counts()['H']
test_games_df.side.value_counts()['A']
# Both 1620

# Two games do not have a home side and an away side, which was messing with the X_train y_train assignment
# Fixing that here by dropping those games

# All the home IDs
home_ids = train_games_df.loc[train_games_df.side == 'H'].id.values
# All the away IDs
away_ids = train_games_df.loc[train_games_df.side == 'A'].id.values

# IDs missing home values
missing_home = set(away_ids) - set(home_ids)

for id in missing_home:
    train_games_df.drop(train_games_df.loc[train_games_df['id'] == id].index, inplace=True)
    train_ids.remove(id)

In [81]:
# Did it work?
print(train_games_df.side.value_counts()['A']
,train_games_df.side.value_counts()['H'], len(train_ids))

6476 6476 6476


In [82]:
pd.options.mode.chained_assignment = None  # default='warn'

X_train = []
y_train = []

counter = 0

for id in train_ids:
    temp = train_games_df.loc[train_games_df["id"] == id]
    temp_home = temp.loc[temp["side"] == 'H']
    temp_away = temp.loc[temp["side"] == 'A']

    y_train.append(np.array([temp_home["points"], temp_away["points"]]))
    temp_home.drop(["side", "points"], axis=1, inplace=True)
    temp_away.drop(["side", "points"], axis=1, inplace=True)

    temp_home = temp_home.to_numpy()
    temp_away = temp_away.to_numpy()

    counter = counter + 1

    if np.shape(temp_home) == (1, 30) and np.shape(temp_away) == (1, 30):
        X_train.append(np.concatenate([temp_home, temp_away], axis=1)[0])
    else:
        print(counter)
        print(temp_home)
        print(temp_away)
        continue
    # try:
    #     X_train.append(np.concatenate([temp_home, temp_away], axis=1)[0])
    #     print(np.shape(temp_home))
    #     print(np.shape(temp_away))
    # except:
    #     print(np.shape(temp_home))
    #     print(np.shape(temp_away))


X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1, 2)

In [13]:
X_train

[array([2.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.20000000e+01,
        3.12500000e-01, 2.58000000e+02, 1.45000000e+02, 3.70000000e+00,
        1.13000000e+02, 2.50000000e+01, 4.50000000e+00, 2.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 1.56300000e+03, 4.14000000e+02,
        1.18000000e+01, 2.09000000e+02, 5.00000000e+00, 5.00000000e+00,
        2.00000000e+00, 5.20000000e+01, 5.00000000e+00, 1.60000000e+01,
        2.00000000e+00, 2.00000000e+00, 2.20000000e+01, 3.90000000e+01,
        5.64102564e-01, 0.00000000e+00, 2.00000000e+00, 5.00000000e+00,
        2.70000000e+01, 8.00000000e-01, 6.23000000e+02, 4.14000000e+02,
        1.18000000e+01, 2.09000000e+02, 4.20000000e+01, 5.00000000e+00,
        3.00000000e+00, 1.00000000e+00, 2.00000000e+00, 2.03700000e+03,
        1.45000000e+02, 3.70000000e+00, 1.13000000e+02, 4.50000000e+00,
        0.00000000e+00, 1.00000000e+00, 1.40000000e+01, 1.20000000e+01,
        1.50000000e+01, 0.00000000e+00, 0.00000000e+00, 2.500000

In [44]:
for id in train_ids:
    print(id)

    401112083

401112083
401405122
400787321
401282706
401416590
400869409
401112143
400547984
401013350
401531915
400868932
401426546
401532404
401015000
400945282
400938670
400548326
401112437
400787454
400548131
401112087
401531373
401415213
400945000
401403940
401248781
401411092
401020761
400548263
400945301
401524000
400787333
401114277
400548068
401404089
401207218
401532419
400869547
400945291
401282625
401212478
401014998
400869152
401012812
401525824
401309639
400937455
401260157
400548006
401405144
400945289
401112164
401426337
400934582
401266134
401013356
401114262
400869393
400548173
401112092
401525558
400869440
401132974
400768337
401287912
400787358
400869232
401117516
400869697
400868920
400869109
401117497
400945297
401013101
400869296
401234559
401112157
401426349
401405140
400944876
401014991
400547727
401531841
401416619
401012336
401414783
401403999
401520233
401520235
400548016
401013158
400763502
400548249
401012792
401310742
400763616
400938594
401117883
401114337
401114202
